# Review: V3 Revised FORMALIZE_INCORRECTLY Cases

The revised V3 prompt classified 21/27 cases as FORMALIZE_INCORRECTLY. Is this correct?

In [ ]:
import pandas as pd
from pathlib import Path

v3 = pd.read_csv('../results/error_analysis/gpt-5_folio_baseline_20251223_011844_v3_revised.csv')
responses_dir = Path('../results/simplelean/gpt-5_folio_baseline_20251223_011844/responses')

fi_cases = v3[v3['root_cause_category'] == 'FORMALIZE_INCORRECTLY'].copy()
print(f"FORMALIZE_INCORRECTLY cases: {len(fi_cases)}")

## Case-by-Case Review

For each case, check:
1. Is this truly a formalization error?
2. Or should it be AXIOMATIZE_CONTRADICTION, PROOF_INCOMPLETE, etc.?

In [ ]:
def show_case(case_idx):
    row = fi_cases[fi_cases['case_idx'] == case_idx].iloc[0]
    response_path = responses_dir / f'case_{case_idx}.txt'
    response = response_path.read_text() if response_path.exists() else 'Not found'
    
    print(f"=" * 80)
    print(f"Case {case_idx} | {row['pattern']}")
    print(f"=" * 80)
    print(f"\nV3 Error Description: {row['error_description']}")
    print(f"\nConclusion: {row['conclusion']}")
    print(f"\n--- Lean Code ---")
    print(response)
    print()

In [ ]:
# Case 77 - Classifier says FORMALIZE_INCORRECTLY but describes contradiction
show_case(77)

**Case 77 Analysis:**
- V3 says: "if Hannah needs money, she does not need money"
- This is a CONTRADICTION (`Needs → ¬Needs`), not a formalization error
- Should be: **AXIOMATIZE_CONTRADICTION**

In [ ]:
# Case 83 - Classifier says FORMALIZE_INCORRECTLY but describes contradiction
show_case(83)

**Case 83 Analysis:**
- V3 says: "inherently contradictory"
- If it's inherently contradictory, should be **AXIOMATIZE_CONTRADICTION**

In [ ]:
# Case 36 - Classic wrong implication direction
show_case(36)

**Case 36 Analysis:**
- V3 says: "writing a music piece implies being a composer, not the other way around"
- This IS a formalization error - wrong implication direction
- Classification: **FORMALIZE_INCORRECTLY** (correct)

In [ ]:
# Case 118 - Wrong direction for location
show_case(118)

**Case 118 Analysis:**
- V3 says: "reverses the intended relationship"
- This IS a formalization error - wrong argument order
- Classification: **FORMALIZE_INCORRECTLY** (correct)

In [ ]:
# Show all case indices for manual review
print("All FORMALIZE_INCORRECTLY case indices:")
print(fi_cases['case_idx'].tolist())

## Summary: Misclassifications Found

| Case | V3 Says | Should Be | Reason |
|------|---------|-----------|--------|
| 77 | FORMALIZE_INCORRECTLY | AXIOMATIZE_CONTRADICTION | Describes `Needs → ¬Needs` |
| 83 | FORMALIZE_INCORRECTLY | AXIOMATIZE_CONTRADICTION | Describes "inherently contradictory" |

**Issue**: V3 prompt confuses "axiom that creates contradiction" with "incorrectly formalized axiom"

In [ ]:
# Review remaining cases quickly
for case_idx in fi_cases['case_idx'].tolist():
    if case_idx not in [77, 83, 36, 118]:
        row = fi_cases[fi_cases['case_idx'] == case_idx].iloc[0]
        print(f"Case {case_idx} | {row['pattern']}")
        print(f"  {row['error_description'][:100]}...")
        print()